# NLP with CNN

### Exercise objectives:

- Use CNN instead of RNN for NLP

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras import layers, Sequential

In [1]:
######################################
### Run this cell to load the data ###
######################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2023-04-22 13:42:23.799137: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFXPFFC/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFXPFFC/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2023-04-22 13:42:50.693380: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Remember that to do NLP, you need to go through one of the following options, as shown here: 

<img src="embedding_or_RNN.png" width="700px" />

But in both cases, you can replace the recurrent layer (top part) by a CNN layer. We will go into both, starting from the one on the left were the embedding is learned in the network.




# Part 1 : Concatenate a Keras Embedding with a Conv1D🔥 

Let's train a fancy network here. 

Each of your words is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). So, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. As shown here

<img src="image_comparison.png" width="500px" />

Well, in that case, as these are close to images, why not using convolution on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2 dimensional as the filters can move up and down, and left and right. In the case of our sentences, we want the kernel to move _only_ in the word by word direction (The alternative, moving coordinate by coordinate of the embedding space doesn't make much sense).

So let's create a model that use convolutions.

## First, the data

❓ **Question** ❓ You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [6]:
# YOUR CODE HERE
tk = Tokenizer()
tk.fit_on_texts(X_train)
X_train_tokenized = tk.texts_to_sequences(X_train)
X_test_tokenized = tk.texts_to_sequences(X_test)

In [8]:
X_train_pad = pad_sequences(X_train_tokenized, maxlen=150, padding='post', dtype='float32')
X_test_pad = pad_sequences(X_test_tokenized, maxlen=150, padding='post', dtype='float32')

In [10]:
vocab_size = len(tk.word_index)
vocab_size

30419

## Using 1D Convolution.

❓ **Question** ❓ Define a model that has :
- an `Embedding` layer: `input_dim` is the `vocab_size + 1`, `output_dim` is the embedding space dimension, and `mask_zero` has to be set to `True`. Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly

❗ **Remark** ❗ The size of the `Conv1D` kernel corresponds exactly to the number of side-by-side words (tokens) each kernel is taking into account ;)

In [17]:
# YOUR CODE HERE
model_cnn = Sequential()
model_cnn.add(layers.Embedding(input_dim=vocab_size+1, input_length=150, output_dim=32, mask_zero=True))
model_cnn.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(16, activation='relu'))
model_cnn.add(layers.Dense(1, activation='sigmoid'))

model_cnn.compile(optimizer='rmsprop', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

❓ **Question** ❓ Look at the number of parameters. You can compare it to the model that you had in previous exercise (esp. the first one)

In [18]:
# YOUR CODE HERE
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 150, 32)           973440    
                                                                 
 conv1d_2 (Conv1D)           (None, 148, 32)           3104      
                                                                 
 flatten_2 (Flatten)         (None, 4736)              0         
                                                                 
 dense_4 (Dense)             (None, 16)                75792     
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,052,353
Trainable params: 1,052,353
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_cnn.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 5ms/step - loss: 0.6945 - accuracy: 0.5120 - val_loss: 0.6907 - val_accuracy: 0.5733
Epoch 2/20
55/55 [==============================] - 0s 3ms/step - loss: 0.5839 - accuracy: 0.7731 - val_loss: 0.6033 - val_accuracy: 0.7093
Epoch 3/20
55/55 [==============================] - 0s 3ms/step - loss: 0.2945 - accuracy: 0.9326 - val_loss: 0.4777 - val_accuracy: 0.7773
Epoch 4/20
55/55 [==============================] - 0s 3ms/step - loss: 0.0930 - accuracy: 0.9886 - val_loss: 0.4186 - val_accuracy: 0.8000
Epoch 5/20
55/55 [==============================] - 0s 4ms/step - loss: 0.0236 - accuracy: 0.9977 - val_loss: 0.4882 - val_accuracy: 0.7747
Epoch 6/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.4592 - val_accuracy: 0.7987
Epoch 7/20
55/55 [==============================] - 0s 4ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.4494 - val_accuracy: 0.8307
Epoch 8/20
55/55 [==

# Part 2 : Learn a Word2Vec representation, and then feed it into a NN with a `Conv1D`🔥 

In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network, which was the CNN counterpart of the left part of this Figure: 

<img src="embedding_or_RNN.png" width="700px" />

Now, let's try to replace the RNN with a CNN for an architecture as on the right side.

❓ **Question** ❓ Learn a word2vec model or load a trained one directly from GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long, it prepares you for real world challenges - but you have already done all the building bricks in the previous exercises

In [ ]:
# YOUR CODE HERE


❓ **Question** ❓ Now construct a model that has a `Conv1D` layer, a flatten layer, a dense layer, and an output layer. Compile it, and fit it on the train data. You can then evaluate it on the test set.

In [ ]:
# YOUR CODE HERE

❓ **Question** ❓ You might be frustrated by the accuracy you got, this happens to us all at some point. Once you have tested your first iteration, you need to improve your models: by making them more complex, changing the parameters, stacking additional layers, and so on.

Only practice and experimentation will get you there. So you can go back to your previous models, change them and try to get better results ;)